Open initial condition file. Download from olympus if necessary.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import holoviews as hv
hv.extension('bokeh', 'matplotlib')

In [ ]:
url = "http://atmos.washington.edu/~nbren12/data/ic.nc"
![[ -e ic.nc ]] || wget {url}
ic = xr.open_dataset("ic.nc")

In [ ]:
from sam.case import InitialConditionCase
import tempfile, os, shutil
from os.path import join

def process_ngaqua(ic):
    """Diagnose the vertical velocity using SAM's pressure solver"""

    path = tempfile.mkdtemp(dir=os.getcwd(), prefix='.tmp')
    case = InitialConditionCase(ic=ic, path=path)
    case.prm['parameters']['nstop'] = 0
    case.prm['parameters']['dodamping'] = True
    case.prm['parameters']['dosgs'] = True
    
    case.save()

    !docker run -v {case.path}:/run -w /run  nbren12/uwnet ./run.sh
    !docker run -v {case.path}:/run -w /run/OUT_3D  nbren12/uwnet bin3D2nc CASE__1_0000000000.bin3D > /dev/null
    processed = xr.open_dataset(join(case.path, 'OUT_3D', 'CASE__1_0000000000.nc'))
    shutil.rmtree(case.path)
    
    return processed

# Blurring the initial condition

V is staggered to the bottom and U to the right.

In [ ]:
v = ic.V.values

In [ ]:
from scipy.ndimage import gaussian_filter1d

def pad_south_v(v, n):
    v[:,0,:] = 0 
    return np.pad(v, [(0,0), (n,0), (0,0)], mode='reflect', reflect_type='odd')

def pad_north_v(v, n):
    
    v = np.pad(v, [(0,0), (0,1), (0,0)], mode='constant')
    return np.pad(v, [(0,0), (0,n-1), (0,0)], mode='reflect', reflect_type='odd')

def pad_v(v,n):
    return pad_north_v(pad_south_v(v, n), n)

def blur_staggered_y(v, sigma, n=None):
    if n is None:
        n = max(int(3*sigma), 3)
    v_pad = pad_v(v, n)
    return gaussian_filter1d(
        gaussian_filter1d(v_pad, sigma, axis=-2)[:,n:-n,:],
        sigma,
        mode='wrap', axis=-1)
    
def blur_centered_y(f, sigma):
    return gaussian_filter1d(
        gaussian_filter1d(f, sigma, axis=-2, mode='nearest'),
        sigma,
        mode='wrap', axis=-1)


class BlurXarray:
    
    def blur_staggered_y(V, sigma):
        return xr.apply_ufunc(blur_staggered_y, V,
                   input_core_dims=[['z', 'y' ,'x']],
                   output_core_dims=[['z', 'y' ,'x']],
                   kwargs=dict(sigma=sigma))
    
    def blur_centered_y(U, sigma):
        return xr.apply_ufunc(blur_centered_y, U,
                   input_core_dims=[['z', 'y' ,'x']],
                   output_core_dims=[['z', 'y' ,'x']],
                   kwargs=dict(sigma=sigma))
    
    @classmethod
    def blur(cls, ds, sigma):
        data = {}
        for key in ds.data_vars:
            if {'x','y'} < set(ds[key].dims):
                if key == 'V':
                    data[key] = cls.blur_staggered_y(ds[key], sigma)
                else:
                    data[key] =  cls.blur_centered_y(ds[key], sigma)
            else:
                data[key] = ds[key]
        return xr.Dataset(data)

In [ ]:
np.testing.assert_array_equal(pad_v(v, 10)[:,10:-10,:], v)

In [ ]:
plt.pcolormesh(blur_staggered_y(v, sigma=1.0)[5])

In [ ]:
plt.pcolormesh(blur_centered_y(ic.U.values, sigma=1.0)[5])

In [ ]:
u_b = BlurXarray.blur_centered_y(ic.U, 1)
v_b = BlurXarray.blur_staggered_y(ic.V, 1)

In [ ]:
ic_b = BlurXarray.blur(ic, 1.0)


fig, (a,b) = plt.subplots(2, 1)
ic_b.V[5].plot(ax=a)
ic_b.U[5].plot(ax=b)

In [ ]:
ic_b_n = process_ngaqua(ic_b)

In [ ]:
ic_b_n.W[0,12].plot()

In [ ]:
%%output size=200
%%opts Image[width=300, height=150](cmap='RdBu_r') {-framewise -axiswise}
ds = hv.Dataset(ic_b_n.W[0])
ds.to.image(["x", "y"])

## Run with blurred initial condition

In [ ]:
86400 / 100

In [ ]:
# path = tempfile.mkdtemp(dir=os.getcwd(), prefix='.tmp')
case = InitialConditionCase(ic=ic_b, path=path)
case.prm['parameters']['dt'] = 100
case.prm['parameters']['nstop'] = 864
case.prm['parameters']['dodamping'] = True
case.prm['parameters']['dosgs'] = True
case.prm['parameters']['nstatfrq'] = 1
case.prm['parameters']['nstat'] = 36
case.prm['parameters']['nsave3d'] = 78
case.prm['parameters']['nsave2d'] = 78
case.prm['parameters']['khyp'] = 1e17


case.save()

!docker run -v {case.path}:/run -w /run  nbren12/uwnet ./run.sh
# !docker run -v {case.path}:/run -w /run/OUT_3D  nbren12/uwnet bin3D2nc CASE__1_0000000000.bin3D > /dev/null
# processed = xr.open_dataset(join(case.path, 'OUT_3D', 'CASE__1_0000000000.nc'))
# shutil.rmtree(case.path)
!docker run -v {case.path}:/run -w /run  nbren12/uwnet /opt/sam/docker/convert_files.sh > /dev/null

In [ ]:
ds_3d = xr.open_mfdataset(f"{path}/OUT_3D/*.nc")

Here is the zonal mean meridional winds. The response looks similar to the simulations with non-blurred initial conditions.

In [ ]:
ds_3d.V.mean('x').plot(col='time', col_wrap=4)

In [ ]:
ds_3d.V.isel(x=0).plot(col='time', col_wrap=4)

And for a single slice

Here is the vertical velocity. It looks like there is some kind of wave which starts at the center of the domain and propagates outward very rapidly (at around 50 m /s). I am not sure if this is the kind of response one would expect from turning of the diabatic forcing suddenly.

In [ ]:
ds_3d.W.mean('x').plot(col='time', col_wrap=4)

This propogation is even more clear in this hovmoller diagram  zonally averaged vertical vecloity at z=7763 m.

In [ ]:
ds_3d.W.isel(z=16).mean('x').plot(y='y')

Here is the zonally averaged temperature.

In [ ]:

levels = np.r_[:11]*10+273
ds_3d.SLI.mean('x').plot.contourf(col='time', col_wrap=4, levels=levels)

These are snapshots of $W$ at z = 6555.

In [ ]:
ds_3d.W[:,15].plot(col='time', col_wrap=4)

And V

In [ ]:
ds_3d.V[:,15].plot(col='time', col_wrap=4)

The hyperdiffusivity is fairly high.

In [ ]:
! grep -i khyp {path}/CASE/*.nml 

# Conclusions

Blurring the initial conditions does not make the meridional velocity fields behave better. 

$ b_t - N^2 v_y =  S$

$ v_t + b_y = 0$
